### Pothole/Non-Pothole Data Processing

Description: stiches sensor and pothole files together into a new combined file. Breaks up data into time intervals.

In [34]:
import pandas as pd
pd.options.display.max_rows = 100
pd.set_option('display.precision', 10)  #for displaying timestamps

In [38]:
sensorsFilePath = 'data/Pothole_Non_Pothole/trip6_sensors.csv'
potholesFilePath = 'data/Pothole_Non_Pothole/trip6_potholes.csv'
combinedFilePath = 'data/Pothole_Non_Pothole/trip6_intervals.csv'
sensorsDF = pd.read_csv(sensorsFilePath)
potholesDF = pd.read_csv(potholesFilePath)

#### Sensor Data

In [39]:
sensorsDF.head()

,timestamp,latitude,longitude,speed,accelerometerX,accelerometerY,accelerometerZ,gyroX,gyroY,gyroZ
0,1.4930035626e+09,40.4475257722,-79.9441930614,0.0,0.2176666260,-0.9800415039,0.1583404541,0.0329595480,0.0487339837,0.1078993031
1,1.4930035627e+09,40.4475257722,-79.9441930614,0.0,0.0113525391,-0.9469299316,0.2120971680,0.0235993357,-0.0873756522,0.0175265294
2,1.4930035629e+09,40.4475257722,-79.9441930614,0.0,0.0236511230,-0.9569549561,0.2081146240,-0.0095865810,0.0053814496,0.0030866037
3,1.4930035631e+09,40.4475257822,-79.9441930619,0.0,0.0610809326,-0.9560852051,0.2014617920,-0.0148543136,0.0011598067,-0.0033537188
4,1.4930035633e+09,40.4475257822,-79.9441930619,0.0,0.0701599121,-0.9564666748,0.1945037842,-0.0127994185,-0.0009803096,0.0009376990


#### Pothole Data

In [40]:
potholesDF.head()

,timestamp
0,1.4930036857e+09
1,1.4930036867e+09
2,1.4930036926e+09
3,1.4930037090e+09
4,1.4930037140e+09


#### Grouping points into time intervals with combined sensor/pothole data

Each interval will represent 2 seconds (10 points) of data. They will also contain the following aggregate statistics for points in that interval:

- meanSpeed: mean speed in interval
- sdSpeed: standard deviation of speed in interval
- maxAccelX, maxAccelY, maxAccelZ: maximum value of accelerometerX, accelerometerY, accelerometerY
- maxGyroX, maxGyroY, maxGyroZ: maximum value of gyroX, gyroY, gyroZ
- minAccelX, minAccelY, minAccelZ: minimum value of accelerometerX, accelerometerY, accelerometerY
- minGyroX, minGyroY, minGyroZ: minimum value of gyroX, gyroY, gyroZ
- meanAccelX, meanAccelY, meanAccelZ: mean of accelerometerX, accelerometerY, accelerometerY
- meanGyroX, meanGyroY, meanGyroZ: mean of gyroX, gyroY, gyroZ
- sdAccelX, sdAccelY, sdAccelZ: standard deviation of accelerometerX, accelerometerY, accelerometerY
- sdGyroX, sdGyroY, sdGyroZ: standard deviation of gyroX, gyroY, gyroZ
- pothole (True/False): whether a pothole was encountered in that interval

In [41]:
# Time window (2 seconds) in number of points
# 5 points = 1 second
window = 10

def intervalHasPothole(intervalStart, intervalEnd, potholesDF):
    potholeTimestamps = potholesDF['timestamp']
    for index, potholeTime in potholeTimestamps.iteritems():
        if intervalStart < potholeTime and potholeTime <= intervalEnd:
            return True
    return False

intervalsDF = pd.DataFrame(columns=['meanSpeed', 'sdSpeed', 'maxAccelX', 'maxAccelY', 'maxAccelZ', 
                                    'maxGyroX', 'maxGyroY', 'maxGyroZ', 'minAccelX', 'minAccelY', 
                                    'minAccelZ', 'minGyroX', 'minGyroY', 'minGyroZ', 'meanAccelX', 
                                    'meanAccelY', 'meanAccelZ', 'meanGyroX', 'meanGyroY', 'meanGyroZ', 
                                    'sdAccelX', 'sdAccelY', 'sdAccelZ', 'sdGyroX', 'sdGyroY', 
                                    'sdGyroZ', 'pothole'])
for i in xrange(0, len(sensorsDF), window):
    intervalIndex = len(intervalsDF)
    interval = sensorsDF[i:i+10]
    intervalStart = list(interval['timestamp'])[0]
    intervalEnd = list(interval['timestamp'])[-1]
    meanSpeed, sdSpeed = interval['speed'].mean(), interval['speed'].std()
    maxAccelX, minAccelX = interval['accelerometerX'].max(), interval['accelerometerX'].min()
    maxAccelY, minAccelY = interval['accelerometerY'].max(), interval['accelerometerY'].min()
    maxAccelZ, minAccelZ = interval['accelerometerZ'].max(), interval['accelerometerZ'].min()
    maxGyroX, minGyroX = interval['gyroX'].max(), interval['gyroX'].min()
    maxGyroY, minGyroY = interval['gyroY'].max(), interval['gyroY'].min()
    maxGyroZ, minGyroZ = interval['gyroZ'].max(), interval['gyroZ'].min()
    meanAccelX, sdAccelX = interval['accelerometerX'].mean(), interval['accelerometerX'].std()
    meanAccelY, sdAccelY = interval['accelerometerY'].mean(), interval['accelerometerY'].std()
    meanAccelZ, sdAccelZ = interval['accelerometerZ'].mean(), interval['accelerometerZ'].std()
    meanGyroX, sdGyroX = interval['gyroX'].mean(), interval['gyroX'].std()
    meanGyroY, sdGyroY = interval['gyroY'].mean(), interval['gyroY'].std()
    meanGyroZ, sdGyroZ = interval['gyroZ'].mean(), interval['gyroZ'].std()
    pothole = intervalHasPothole(intervalStart, intervalEnd, potholesDF)
    intervalSummary = [meanSpeed, sdSpeed, maxAccelX, maxAccelY, maxAccelZ, maxGyroX, maxGyroY, 
                       maxGyroZ, minAccelX, minAccelY, minAccelZ, minGyroX, minGyroY, minGyroZ, 
                       meanAccelX, meanAccelY, meanAccelZ, meanGyroX, meanGyroY, meanGyroZ,
                       sdAccelX, sdAccelY, sdAccelZ, sdGyroX, sdGyroY, sdGyroZ, pothole]
    intervalsDF.loc[intervalIndex] = intervalSummary
    
intervalsDF.head()

,meanSpeed,sdSpeed,maxAccelX,maxAccelY,maxAccelZ,maxGyroX,maxGyroY,maxGyroZ,minAccelX,minAccelY,...,meanGyroX,meanGyroY,meanGyroZ,sdAccelX,sdAccelY,sdAccelZ,sdGyroX,sdGyroY,sdGyroZ,pothole
0,0.0000000000,0.0000000000,0.2176666260,-0.9469299316,0.2120971680,0.0329595480,0.0487339837,0.1078993031,0.0113525391,-0.9800415039,...,-0.0043633764,-0.0046499325,0.0127618946,0.0557300765,0.0087466703,0.0144098904,0.0174007225,0.0331251346,0.0339172088,False
1,0.0479999989,0.1011928829,0.0785064697,-0.9331359863,0.2717742920,-0.0031795480,0.0096606169,0.0072680329,0.0588684082,-0.9709320068,...,-0.0141437556,-0.0005465073,-0.0017427194,0.0069277981,0.0133113464,0.0490568595,0.0071068851,0.0053237358,0.0050463825,False
2,0.2259999931,0.0800277703,0.0674438477,-0.9461822510,0.2098083496,-0.0063337960,0.0054416371,0.0020445088,0.0550079346,-0.9678955078,...,-0.0132972965,-0.0008710934,-0.0015199726,0.0045312778,0.0070150467,0.0209263038,0.0036947555,0.0040217262,0.0025294318,False
3,0.0569999993,0.1802498244,0.1011352539,-0.9107055664,0.3683624268,0.0032314797,0.0086475504,0.0052911685,0.0664672852,-0.9496612549,...,-0.0090338419,-0.0018563032,-0.0023267506,0.0088680387,0.0120398286,0.0522918066,0.0081263068,0.0063212338,0.0049780064,False
4,1.2889999986,0.6616889319,0.1100006104,-0.8900909424,0.3652343750,-0.0053862433,0.0597397633,0.0158157148,0.0283508301,-0.9397277832,...,-0.0178772678,0.0196877912,-0.0057781541,0.0243626337,0.0170524731,0.0215171973,0.0065494289,0.0226486220,0.0163744159,False


#### Intervals with potholes

In [42]:
intervalsDF[intervalsDF['pothole'] == True]

,meanSpeed,sdSpeed,maxAccelX,maxAccelY,maxAccelZ,maxGyroX,maxGyroY,maxGyroZ,minAccelX,minAccelY,...,meanGyroX,meanGyroY,meanGyroZ,sdAccelX,sdAccelY,sdAccelZ,sdGyroX,sdGyroY,sdGyroZ,pothole
61,9.6079998970,0.7463507144,0.3191528320,-0.6574707031,0.6968536377,0.1304916976,0.3210717663,0.1497466186,-0.7182464600,-1.3377685547,...,0.0286588890,0.0549089222,-0.0330075381,0.3189966289,0.2313527210,0.1523701298,0.0915202664,0.2103093672,0.1661105470,True
62,11.5019997597,0.5442591529,0.4201660156,-0.8198394775,0.6541442871,0.0927733471,0.6253592261,0.1302813079,-0.3746490479,-1.2853546143,...,-0.0563687872,0.0367370289,-0.0550488447,0.1945242572,0.1340302622,0.1228299154,0.1610075926,0.2283527206,0.2272700085,True
65,13.7519997597,0.2247366689,0.1572875977,-0.7302551270,0.3397369385,0.3867916578,0.2053102790,0.3228387737,-0.1331787109,-1.0730285645,...,-0.0074892884,-0.0279241495,0.0183342662,0.0842193626,0.1124882588,0.1092399833,0.1828185347,0.1327779127,0.1268884029,True
73,11.7749997139,0.6354569821,0.5521087646,-0.7353668213,0.5048828125,0.6640530925,0.2008843716,0.5188905731,-0.6811828613,-1.0848083496,...,0.0882258398,-0.1600026118,0.1418432087,0.3440100626,0.1270343330,0.0902955121,0.2291065399,0.2967643671,0.2459798362,True
75,13.6869999886,0.3349976438,0.6703491211,-0.7319183350,0.6246643066,0.6866204533,0.6902575324,0.1733624659,-0.8412780762,-1.0897674561,...,0.0438787215,0.0339555702,-0.0670231094,0.3664967171,0.1070546572,0.0947587328,0.2565079154,0.2919712530,0.2426217373,True
76,15.4270003319,0.7250447549,0.8647766113,-0.5925292969,0.4685821533,0.3289283578,0.3860012316,0.5476415276,-0.7893829346,-1.2871398926,...,-0.0245879810,-0.0371737607,0.0176499670,0.4312860428,0.1883023337,0.1619944648,0.1814881912,0.2984256896,0.2925179091,True
77,16.7980001450,0.1394275821,0.5782165527,-0.4690856934,0.6850280762,0.2723574899,0.7928491555,0.2537073728,-0.4995727539,-1.8171997070,...,-0.0581144094,0.2337382187,-0.1230664583,0.3048030020,0.3612550680,0.2315758112,0.2555924534,0.2863408018,0.1824868710,True
78,15.1100003242,1.6310870475,0.1742248535,-0.9388580322,0.1182708740,0.1890745837,0.0537247476,0.2211267927,-0.2343292236,-1.2772216797,...,-0.0009630257,-0.0399843011,0.0328260970,0.1345599830,0.1104204978,0.0662914828,0.0849346060,0.0985365618,0.0880066292,True
80,7.3979997635,1.0224673670,0.4824676514,-0.6853637695,0.7453918457,0.1667258685,0.6318067391,0.3324674326,-0.5041351318,-1.1439056396,...,0.0018404840,0.0759264297,-0.0693743877,0.2740424441,0.1349041061,0.1065849686,0.1066668312,0.2872687266,0.2769360018,True
82,16.0609994888,1.0029556957,0.8315277100,-0.5032806396,0.5885467529,0.1831077713,0.4759843848,0.2338279406,-0.3936309814,-1.2778930664,...,-0.0006775614,0.0143625875,-0.0126253277,0.4260477774,0.2156084985,0.2016259649,0.1749885477,0.2703476917,0.1986334967,True


#### Save to CSV

In [43]:
intervalsDF.to_csv(combinedFilePath)

### Road Conditions (Good Road/Bad Road) Data Processing

Description: Breaks up road condtions data into time intervals. Only contains sensor, not pothole data. Labels are: 0 (good road) and 1 (bad road)

In [92]:
sensorsFilePath = 'data/Good_Road_Bad_Road/4_29_roadcond_sensors_GOOD_t11.csv'
intervalsFilePath = 'data/Good_Road_Bad_Road/good10_intervals.csv'
label = 0
sensorsDF = pd.read_csv(sensorsFilePath)
sensorsDF.head()

,timestamp,latitude,longitude,speed,accelerometerX,accelerometerY,accelerometerZ,gyroX,gyroY,gyroZ
0,1.4934795859e+09,40.4624712607,-79.9277117104,7.1999998093,-0.0306854248,-0.9515380859,0.2197723389,0.0063657540,-0.0011049455,0.0011342403
1,1.4934795860e+09,40.4624712607,-79.9277117104,7.1999998093,-0.0171966553,-0.9342651367,0.2519531250,-0.0158977401,0.0001025317,-0.0054941013
2,1.4934795862e+09,40.4624712607,-79.9277117104,7.1999998093,-0.0351257324,-0.9477996826,0.2282257080,-0.0129845082,-0.0158703096,0.0063053002
3,1.4934795864e+09,40.4624470370,-79.9276365247,6.7399997711,-0.0448455811,-0.9525451660,0.2569427490,-0.0414939140,0.0023971113,-0.0014972292
4,1.4934795866e+09,40.4624470370,-79.9276365247,6.7399997711,-0.0509033203,-0.9353027344,0.2486724854,0.0206839732,0.0115621139,-0.0264734191


#### Grouping points into time intervals with labels

Each interval will represent 5 seconds (25 points) of data. They will also contain the following aggregate statistics for points in that interval:

- meanSpeed: mean speed in interval
- sdSpeed: standard deviation of speed in interval
- maxAccelX, maxAccelY, maxAccelZ: maximum value of accelerometerX, accelerometerY, accelerometerY
- maxGyroX, maxGyroY, maxGyroZ: maximum value of gyroX, gyroY, gyroZ
- minAccelX, minAccelY, minAccelZ: minimum value of accelerometerX, accelerometerY, accelerometerY
- minGyroX, minGyroY, minGyroZ: minimum value of gyroX, gyroY, gyroZ
- meanAccelX, meanAccelY, meanAccelZ: mean of accelerometerX, accelerometerY, accelerometerY
- meanGyroX, meanGyroY, meanGyroZ: mean of gyroX, gyroY, gyroZ
- sdAccelX, sdAccelY, sdAccelZ: standard deviation of accelerometerX, accelerometerY, accelerometerY
- sdGyroX, sdGyroY, sdGyroZ: standard deviation of gyroX, gyroY, gyroZ
- condition (1/0): whether that road is in good (0) or bad (1) condition

In [93]:
# Time window (5 seconds) in number of points
# 5 points = 1 second
window = 25

intervalsDF = pd.DataFrame(columns=['meanSpeed', 'sdSpeed', 'maxAccelX', 'maxAccelY', 'maxAccelZ', 
                                    'maxGyroX', 'maxGyroY', 'maxGyroZ', 'minAccelX', 'minAccelY', 
                                    'minAccelZ', 'minGyroX', 'minGyroY', 'minGyroZ', 'meanAccelX', 
                                    'meanAccelY', 'meanAccelZ', 'meanGyroX', 'meanGyroY', 'meanGyroZ', 
                                    'sdAccelX', 'sdAccelY', 'sdAccelZ', 'sdGyroX', 'sdGyroY', 
                                    'sdGyroZ', 'condition'])
for i in xrange(0, len(sensorsDF), window):
    intervalIndex = len(intervalsDF)
    interval = sensorsDF[i:i+10]
    meanSpeed, sdSpeed = interval['speed'].mean(), interval['speed'].std()
    maxAccelX, minAccelX = interval['accelerometerX'].max(), interval['accelerometerX'].min()
    maxAccelY, minAccelY = interval['accelerometerY'].max(), interval['accelerometerY'].min()
    maxAccelZ, minAccelZ = interval['accelerometerZ'].max(), interval['accelerometerZ'].min()
    maxGyroX, minGyroX = interval['gyroX'].max(), interval['gyroX'].min()
    maxGyroY, minGyroY = interval['gyroY'].max(), interval['gyroY'].min()
    maxGyroZ, minGyroZ = interval['gyroZ'].max(), interval['gyroZ'].min()
    meanAccelX, sdAccelX = interval['accelerometerX'].mean(), interval['accelerometerX'].std()
    meanAccelY, sdAccelY = interval['accelerometerY'].mean(), interval['accelerometerY'].std()
    meanAccelZ, sdAccelZ = interval['accelerometerZ'].mean(), interval['accelerometerZ'].std()
    meanGyroX, sdGyroX = interval['gyroX'].mean(), interval['gyroX'].std()
    meanGyroY, sdGyroY = interval['gyroY'].mean(), interval['gyroY'].std()
    meanGyroZ, sdGyroZ = interval['gyroZ'].mean(), interval['gyroZ'].std()
    condition = label  # label for that road
    intervalSummary = [meanSpeed, sdSpeed, maxAccelX, maxAccelY, maxAccelZ, maxGyroX, maxGyroY, 
                       maxGyroZ, minAccelX, minAccelY, minAccelZ, minGyroX, minGyroY, minGyroZ, 
                       meanAccelX, meanAccelY, meanAccelZ, meanGyroX, meanGyroY, meanGyroZ,
                       sdAccelX, sdAccelY, sdAccelZ, sdGyroX, sdGyroY, sdGyroZ, condition]
    intervalsDF.loc[intervalIndex] = intervalSummary

intervalsDF.head()

,meanSpeed,sdSpeed,maxAccelX,maxAccelY,maxAccelZ,maxGyroX,maxGyroY,maxGyroZ,minAccelX,minAccelY,...,meanGyroX,meanGyroY,meanGyroZ,sdAccelX,sdAccelY,sdAccelZ,sdGyroX,sdGyroY,sdGyroZ,condition
0,6.8629998684,0.2335736334,0.0168609619,-0.8897705078,0.4085083008,0.0206839732,0.0202144579,0.0096552905,-0.0719451904,-0.9525451660,...,-0.0165296550,-0.0063826917,-0.0095354483,0.0284329277,0.0212930574,0.0763773856,0.0232140437,0.0173342640,0.0168641370,0.0
1,9.2989999771,0.1467008811,0.0302124023,-0.8407897949,0.3176422119,0.0585330850,0.0043838295,0.0627129163,-0.1067199707,-1.0450286865,...,-0.0368253127,-0.0547510500,0.0229011081,0.0457280379,0.0763773554,0.0405139997,0.0462831883,0.0448275789,0.0343289158,0.0
2,10.1630002022,0.4576036782,0.2244567871,-0.8934173584,0.4047698975,0.0939941401,0.2264693601,0.0707705765,-0.0833129883,-1.0738677979,...,0.0097506051,0.0002575010,0.0112944129,0.1011962751,0.0567204523,0.0509594258,0.0536785009,0.0962725547,0.0480450002,0.0
3,9.7279998779,1.1152458531,0.0686340332,-0.8290557861,0.1425933838,0.0312774954,0.0088073400,0.0359452179,-0.1676483154,-1.0739288330,...,-0.0181984983,-0.0236022652,0.0017937722,0.0686328527,0.0680783136,0.0282995426,0.0409563126,0.0271755681,0.0273302558,0.0
4,2.7789999723,0.2222336480,0.1283111572,-0.8757629395,0.3111419678,0.0229308823,-0.1373626532,0.1865640217,-0.2063751221,-1.1389007568,...,-0.0069536468,-0.3009696138,0.1097792019,0.0998738211,0.0719722968,0.0173475407,0.0209984055,0.1025167542,0.0554998979,0.0


#### Save to CSV

In [94]:
intervalsDF.to_csv(intervalsFilePath)